#### A 3 node experiment to demonstrate the network simulation and attack process in cyborg

In [1]:
import random
from os.path import dirname
from pprint import pprint
import inspect
from CybORG import CybORG
from CybORG.Simulator.Scenarios import FileReaderScenarioGenerator
from CybORG.Simulator.Actions import *


path = str(inspect.getfile(CybORG))
path= dirname(path) + f'/Simulator/Scenarios/scenario_files/Scenario_3nodes.yaml'
path = f'C:/Users/HPP/Desktop/cyborg/vu_cyborg/CybORG/CybORG/Simulator/Scenarios/scenario_files/Scenario_3nodes.yaml'
sg = FileReaderScenarioGenerator(path)
print('Scenario file path is:',path)

env = CybORG(scenario_generator=sg)
#pprint(dir(env))

#pprint('IP map of network is:',env.get_ip_map())
#pprint('Red agent state is:',env.get_agent_state('Red'))
#pprint('Red agent action space is:',env.get_action_space('Red'))
print('==> IP map of network is:')
pprint(env.get_ip_map())



ips= env.get_ip_map()
userip= ips['User0']
print('-> user ip to attack is:',userip)





#results = env.reset(agent='Red')
#agent = B_lineAgent()

#for step in range(30):
#    action = agent.get_action(results.observation,results.action_space)
#    #print('Action is:',action)
#    results = cyborg.step(action=action, agent='Red')
#    #print(results.reward) 

Scenario file path is: C:/Users/HPP/Desktop/cyborg/vu_cyborg/CybORG/CybORG/Simulator/Scenarios/scenario_files/Scenario_3nodes.yaml
==> IP map of network is:
{'User0': IPv4Address('10.0.173.216'),
 'User1': IPv4Address('10.0.173.222'),
 'User_router': IPv4Address('10.0.173.219')}
-> user ip to attack is: 10.0.173.216


C:\Users\HPP\miniconda3\envs\cyborg\lib\site-packages\gym\utils\seeding.py:48: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  "Function `rng.randint(low, [high, size, dtype])` is marked as deprecated "


##### Observed state from the perspective of red agent

In [2]:
print('==> Red Agents observed state  is:')
pprint(env.get_agent_state('Red'))

==> Red Agents observed state  is:
{'User0': {'Interface': [{'IP Address': IPv4Address('10.0.173.216'),
                          'Interface Name': 'eth0',
                          'Subnet': IPv4Network('10.0.173.208/28')}],
           'Processes': [{'PID': 3372, 'Username': 'SYSTEM'}],
           'Sessions': [{'Agent': 'Red',
                         'ID': 0,
                         'PID': 3372,
                         'Timeout': 0,
                         'Type': <SessionType.RED_ABSTRACT_SESSION: 10>,
                         'Username': 'SYSTEM'}],
           'System info': {'Architecture': <Architecture.x64: 2>,
                           'Hostname': 'User0',
                           'OSDistribution': <OperatingSystemDistribution.WINDOWS_SVR_2008: 4>,
                           'OSType': <OperatingSystemType.WINDOWS: 2>,
                           'OSVersion': <OperatingSystemVersion.W6_1_7601: 13>,
                           'position': array([0., 0.])}},
 'success': <Trina

#### Action space to the red agent:

In [3]:
print('==> Red Agent action space is:')
pprint(env.get_action_space('Red'))


==> Red Agent action space is:
{'action': {<class 'CybORG.Simulator.Actions.Action.Sleep'>: True,
            <class 'CybORG.Simulator.Actions.AbstractActions.ExploitRemoteService.ExploitRemoteService'>: True,
            <class 'CybORG.Simulator.Actions.AbstractActions.DiscoverNetworkServices.DiscoverNetworkServices'>: True,
            <class 'CybORG.Simulator.Actions.AbstractActions.DiscoverRemoteSystems.DiscoverRemoteSystems'>: True},
 'agent': {'Red': True},
 'hostname': {'User0': True, 'User1': False, 'User_router': False},
 'ip_address': {IPv4Address('10.0.173.216'): True,
                IPv4Address('10.0.173.219'): False,
                IPv4Address('10.0.173.222'): False},
 'password': {'raspberry': False, 'vagrant': False},
 'port': {21: False, 22: False, 25: False, 80: False, 443: False, 3390: False},
 'process': {1091: False,
             1100: False,
             1101: False,
             1102: False,
             3344: False,
             3368: False,
             3372: 

#### Attack action : DiscoverNetworkServices
It allow the red agent to identify services on a host as a prerequisite for running an exploit.
It Calls the low level action 'PortScan' then modifies the observation. The outcome is all open ports running on the host.  It must be used on a host to successfully run the high level action ExploitRemoteServices. 


In [4]:
#creating and executing port discovery attack on user network 
action = DiscoverNetworkServices(session=0,agent='Red',ip_address=userip)
results = env.step(action=action,agent='Red')
print('==> Red agent state after discover network services attack:')
pprint(results.observation)

==> Red agent state after discover network services attack:
{'10.0.173.216': {'Interface': [{'IP Address': IPv4Address('10.0.173.216')}],
                  'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.173.216'),
                                                  'local_port': 22}]},
                                {'Connections': [{'local_address': IPv4Address('10.0.173.216'),
                                                  'local_port': 21}]}]},
 'success': <TrinaryEnum.TRUE: 1>}


In [18]:
#creating and executing port discovery attack on user network 
action_space = env.get_action_space('Red')
#pprint(action_space)
subnets=action_space["subnet"]
known_subnets = [subnet for subnet in subnets if subnets[subnet]]
subnet = known_subnets[0]

print('Subnet visible to red is:',subnet)
action = DiscoverRemoteSystems(subnet = subnet, session=0,agent='Red')
results = env.step(action=action,agent='Red')
print('==> Red agent state after discover network services attack:')
pprint(results.observation)

Subnet visible to red is: 10.0.173.208/28
==> Red agent state after discover network services attack:
{'10.0.173.216': {'Interface': [{'IP Address': IPv4Address('10.0.173.216'),
                                 'Subnet': IPv4Network('10.0.173.208/28')}]},
 '10.0.173.222': {'Interface': [{'IP Address': IPv4Address('10.0.173.222'),
                                 'Subnet': IPv4Network('10.0.173.208/28')}]},
 'success': <TrinaryEnum.TRUE: 1>}


#### Attack action : ExploitRemoteServices
It allow the red agent to examines the target host and returns a selected applicable escalate action if any, as well as processes that are required to be genuine.

In [19]:
#creating and executing attack on user network (can we attack on router? -dont know)
action = ExploitRemoteService(ip_address=userip,session=0,agent='Red')
results = env.step(action=action,agent='Red')
print('==> Red agent state after exploit attack:')
pprint(results.observation)

==> Red agent state after exploit attack:
{'10.0.173.216': {'Interface': [{'IP Address': IPv4Address('10.0.173.216')}],
                  'Processes': [{'Connections': [{'Status': <ProcessState.OPEN: 2>,
                                                  'local_address': IPv4Address('10.0.173.216'),
                                                  'local_port': 22}],
                                 'Process Type': <ProcessType.SSH: 2>},
                                {'Connections': [{'local_address': IPv4Address('10.0.173.216'),
                                                  'local_port': 51418,
                                                  'remote_address': IPv4Address('10.0.173.216'),
                                                  'remote_port': 22}]},
                                {'Connections': [{'local_address': IPv4Address('10.0.173.216'),
                                                  'local_port': 22,
                                                  'remote_

In [8]:
## to do : Imapct and many things etc... 